### Building models

In [ ]:
from pymgpipe import *
from pkg_resources import resource_filename

taxa_directory = resource_filename("pymgpipe", "resources/miniTaxa/")

# Generating random abundance matrix
taxa = [x.split('.xml.gz')[0] for x in os.listdir(taxa_directory)]
abundances = pd.DataFrame(np.random.randint(250,750,size=(4, 10)),columns=['sample_0'+str(i) for i in range(1,11)],index=taxa)
abundances = abundances.apply(lambda x: x/x.sum(),axis=0)
abundances

In [ ]:
# ----- STEP 1: BUILD COMMUNITY MODELS ----
build_models(
    coverage_file=abundances,
    taxa_dir=taxa_directory,
    out_dir='out',
    diet_fecal_compartments=True,
    coupling_constraints=True,
    remove_reverse_vars_from_lp=True,
    hard_remove=True,
    compress=True,
    threads=-1,
    parallel=True,
    solver='gurobi',
    cobra_type='.xml',
    lp_type='.mps',
    compute_metrics=True,
    sample_prefix='mc',
    force=True,
    # diet='test_diet.csv', <-- you can try uncommenting this, however this diet will result in infeasibility
)

### Simulating communities

In [ ]:
# ----- STEP 2: COMPUTE NMPCS ----
res = compute_nmpcs(
    samples='out/problems/',
    out_dir='out/',
    threshold=0,
    fva_type=FVA_TYPE.REGULAR,
    force=True,
    threads=-1,
    objective_percent=100
)
res.nmpc

### Loading models

In [ ]:
from pymgpipe import *

# Loading a cobra model
cobra_model = load_cobra_model('out/models/mc1.xml.gz')
print(type(cobra_model))

# Loading an optlang model
optlang_model = load_model('out/problems/mc1.mps.7z',solver='gurobi') 
print(type(optlang_model))
# OR 
optlang_model = load_model('out/models/mc1.xml.gz')
print(type(optlang_model))

### Solving models

In [ ]:
from pymgpipe import solve_model, load_model

sol = solve_model(model='out/problems/mc1.mps.7z',solver='gurobi')

# OR 

m = load_model('out/problems/mc1.mps.7z', solver='gurobi')
target_reactions = get_reactions(m,regex='Diet_EX_.*')
sol = solve_model(model=m,reactions=target_reactions)

# OR 

sol = solve_model(model='out/models/mc1.xml.gz',solver='gurobi')

### Adding/Removing Diets

In [ ]:
from pymgpipe import *

get_available_diets()

In [ ]:
model = load_model('out/problems/mc1.mps.gz')

add_diet_to_model(model,'test_diet.csv',check=True)
remove_diet(model)

add_diet_to_model(model,'VegetarianDiet',check=True)
remove_diet(model)